<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
<br>NEXT TONNE - BACKGROUND INDEX</p><br>

In [ ]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# General importings
import math
import numpy             as np
import tables            as tb
import pandas            as pd
import matplotlib.pyplot as plt

# IC stuff
import invisible_cities.core.system_of_units as units

# TONNE stuff
from detector_dimensions  import get_dimensions
from detector_dimensions  import print_dimensions

from initial_activities  import get_radiogenic_activities
from initial_activities  import get_radon_activity
from initial_activities  import get_muon_flux
from initial_activities  import get_muon_flux_error
from initial_activities  import print_initial_activities

from detector_backgrounds import get_radiogenic_background_level
from detector_backgrounds import get_radon_background_level
from detector_backgrounds import get_muon_background_level

from rejection_factors    import get_rejection_factors
from roi_settings         import get_roi_settings


# DETAILS OF ANALYSIS

In [ ]:
# Available detectors: "next_2x2", "next_3x3", "next_hd"
DETECTOR = 'next_hd'

# Available radiogenic background levels: "reference", "probable", "optimistic"
RADIOGENIC_BKGND_LEVEL = 'probable'

# Available Radon background levels: "pessimistic", "optimistic"
RADON_BKGND_LEVEL = 'pessimistic'

# Hosting lab: 'LSC', 'LNGS', 'SNOLAB'
HOSTING_LAB = 'LNGS'

# LOADING DETECTOR DATA

In [ ]:
det_dim = get_dimensions(DETECTOR)

In [ ]:
print_dimensions(DETECTOR)

In [ ]:
radiogenic_act = get_radiogenic_activities(RADIOGENIC_BKGND_LEVEL)
radon_act      = get_radon_activity(RADON_BKGND_LEVEL)
muon_flux      = get_muon_flux(HOSTING_LAB)

In [ ]:
print_initial_activities(RADIOGENIC_BKGND_LEVEL, RADON_BKGND_LEVEL, HOSTING_LAB)

# COMPUTING THE DETECTOR BACKGROUND LEVEL

#### Radiogenic contributions

In principle. only the contributions from the 'READOUT_PLANES', 'FIELD_CAGE' and 'INNER_SHIELDING' are considered.

Backgrounds expressed in Becquerel

In [ ]:
radiogenic_bkgnd = get_radiogenic_background_level(DETECTOR, RADIOGENIC_BKGND_LEVEL)
radiogenic_bkgnd['Total'] = radiogenic_bkgnd['Bi214'] + radiogenic_bkgnd['Tl208']
radiogenic_bkgnd

In [ ]:
tot_bkgnd = radiogenic_bkgnd['Total'].sum()
print(f"*** Radiogenic '{DETECTOR}' background: {tot_bkgnd:.4} Bq.")

tot_Bi = radiogenic_bkgnd['Bi214'].sum()
tot_Tl = radiogenic_bkgnd['Tl208'].sum()
print(f"\nTotal Bi214 background: {tot_Bi:8.4} Bq.  ->  {tot_Bi/tot_bkgnd*100:.4} %")
print(f"Total Tl208 background: {tot_Tl:8.4} Bq.  ->  {tot_Tl/tot_bkgnd*100:.4} %")

tot_READOUT_PLANE = radiogenic_bkgnd.loc['READOUT_PLANE'].Total
tot_FIELD_CAGE    = radiogenic_bkgnd.loc['FIELD_CAGE'].Total
tot_ICS           = radiogenic_bkgnd.loc['INNER_SHIELDING', 'Total']
print(f"\nTotal READOUT_PLANE background: {tot_READOUT_PLANE:8.4} Bq.  ->  {tot_READOUT_PLANE/tot_bkgnd*100:.4} %")
print(f"Total FIELD_CAGE background:    {tot_FIELD_CAGE:8.4} Bq.  ->  {tot_FIELD_CAGE/tot_bkgnd*100:.4} %")
print(f"Total ICS background:           {tot_ICS:8.4} Bq.  ->  {tot_ICS/tot_bkgnd*100:.4} %")


#### Radon contribution

The radon contribution is directly translated to an Bi214 concentration in the central CATHODE

In [ ]:
radon_background = get_radon_background_level(DETECTOR, RADON_BKGND_LEVEL)

In [ ]:
print(f"*** Radon '{DETECTOR}' background: {radon_background/units.Bq:.4} Bq.")

print(f"(It will be treated as Bi214 contamination from the CATHODE)")

#### Muons contribution

Xe137 decay is the only muon contribution considered.

In [ ]:
muon_background, muon_background_error = get_muon_background_level(DETECTOR, HOSTING_LAB)

In [ ]:
print(f"*** Muon '{DETECTOR}' background: {muon_background:.4} Bq.")

print(f"(It will be treated as Xe137 contamination from the ACTIVE)")

# LOADING & SELECTING REJECTION FACTORS

In [ ]:
rej_factors = get_rejection_factors(DETECTOR)
rej_factors

### OPTIONS TO STUDY

In [ ]:
energyRes  = 0.7
spatialDef = '10x10x10'

#### Signal efficiency

In [ ]:
sig_eff = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'bb0nu']
sig_eff

#### Radiogenic rejection_factors

In [ ]:
radiogenic_rejections = rej_factors.loc[pd.IndexSlice[:, energyRes, spatialDef], ['Bi214', 'Tl208']]
radiogenic_rejections.drop('ACTIVE', level=0, axis=0, inplace=True)
radiogenic_rejections.drop('CATHODE', level=0, axis=0, inplace=True)
radiogenic_rejections

#### Radon rejection_factor

As in previos NEXT detectors we assume that the whole amount of Radon contamination is translated to Bi214 contamination from the CATHODE.

In [ ]:
radon_rejection  = rej_factors.loc[pd.IndexSlice['CATHODE', energyRes, spatialDef], 'Bi214']
print(f"Radon rejection factor:       {radon_rejection:.4}")

#### Muons rejection_factors

The muons rejection factor = Xe137_activation * Xe137_rejection.

As the Xe137_activation has been considered previously, the Xe137_rejection factor is the only one applied here.

In [ ]:
Xe137_rejection = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'Xe137']

In [ ]:
print(f"Xe137 rejection factor:       {Xe137_rejection:.4}")

# COMPUTING THE DETECTOR BACKGROUND INDEX

#### Estimating the Bq -> ckky conversion factor 

In [ ]:
secs_in_year    = 60 * 60 * 24 * 365
Xe136_abundance = 0.9026
Xe136_mass_kg   = det_dim['ACTIVE_mass'] * Xe136_abundance / units.kg
ROI_settings    = get_roi_settings(energyRes)

In [ ]:
toCKKY = secs_in_year / Xe136_mass_kg / ((ROI_settings['Emax'] - ROI_settings['Emin']) / units.keV)

In [ ]:
print(f"Xe136 mass: {Xe136_mass_kg:.4} kg.")
print(f"ROI: [{ROI_settings['Emin']/units.keV:.4}, {ROI_settings['Emax']/units.keV:.4}] keV.")
print(f"toCKKY factor: {toCKKY:.4}")

#### Radiogenic background index

In [ ]:
radiogenic_bkgnd_index_Bq = radiogenic_bkgnd[['Bi214', 'Tl208']] * radiogenic_rejections[['Bi214', 'Tl208']]
radiogenic_bkgnd_index_Bq['Total'] = radiogenic_bkgnd_index_Bq['Bi214'].add(radiogenic_bkgnd_index_Bq['Tl208'],
                                                                            fill_value = 0.)
radiogenic_bkgnd_index = radiogenic_bkgnd_index_Bq * toCKKY
radiogenic_bkgnd_index

In [ ]:
tot_radiogenic_bkgnd_index = radiogenic_bkgnd_index['Total'].sum()
print(f"Radiogenic '{DETECTOR}' background index: {tot_radiogenic_bkgnd_index:8.3} ckky")

tot_Bi = radiogenic_bkgnd_index['Bi214'].sum()
tot_Tl = radiogenic_bkgnd_index['Tl208'].sum()
print(f"\nTotal Bi214 background index: {tot_Bi:8.3} Bq.  ->  {tot_Bi/tot_radiogenic_bkgnd_index*100:.4} %")
print(f"Total Tl208 background index: {tot_Tl:8.3} Bq.  ->  {tot_Tl/tot_radiogenic_bkgnd_index*100:.4} %")

tot_READOUT_PLANE = radiogenic_bkgnd_index.loc[pd.IndexSlice['READOUT_PLANE', energyRes, spatialDef]].Total
tot_FIELD_CAGE    = radiogenic_bkgnd_index.loc[pd.IndexSlice['FIELD_CAGE', energyRes, spatialDef]].Total
tot_ICS           = radiogenic_bkgnd_index.loc[pd.IndexSlice['INNER_SHIELDING', energyRes, spatialDef]].Total
print(f"\nTotal READOUT_PLANE background index: {tot_READOUT_PLANE:8.3} ckky.  ->  {tot_READOUT_PLANE/tot_radiogenic_bkgnd_index*100:6.4} %")
print(f"Total FIELD_CAGE    background index: {tot_FIELD_CAGE:8.3} ckky.  ->  {tot_FIELD_CAGE/tot_radiogenic_bkgnd_index*100:6.4} %")
print(f"Total ICS           background index: {tot_ICS:8.3} ckky.  ->  {tot_ICS/tot_radiogenic_bkgnd_index*100:6.4} %")


#### Radon background index

In [ ]:
radon_bkgnd_index_Bq = (radon_background / units.Bq) * radon_rejection
radon_bkgnd_index    = radon_bkgnd_index_Bq * toCKKY

In [ ]:
print(f"Radon '{DETECTOR}' background index: {radon_bkgnd_index:8.3} ckky")

#### Muons background index

In [ ]:
muon_bkgnd_index_Bq = muon_background * Xe137_rejection
muon_bkgnd_index    = muon_bkgnd_index_Bq * toCKKY

In [ ]:
print(f"Muon '{DETECTOR}' background index: {muon_bkgnd_index:8.4} ckky")

### DETECTOR SUMMARY

In [ ]:
bkgnd_index = tot_radiogenic_bkgnd_index + radon_bkgnd_index + muon_bkgnd_index

In [ ]:
print(f"\n           '{DETECTOR}' Xe136 mass: {Xe136_mass_kg:.4} Kg")
print(f"\n           '{DETECTOR}' efficiency: {sig_eff}")

print(f"\nRadiogenic '{DETECTOR}' background index: {tot_radiogenic_bkgnd_index:8.3} ckky")
print(f"Radon      '{DETECTOR}' background index: {radon_bkgnd_index:8.3} ckky")
print(f"Muons      '{DETECTOR}' background index: {muon_bkgnd_index:8.3} ckky")

print(f"\nTotal      '{DETECTOR}' background index: {bkgnd_index:8.3} ckky")

# SENSITIVITY

In [ ]:
#from pybbsens import isotope